In [35]:
from PyPDF2 import PdfReader,PdfWriter
import re
import pandas as pd

In [36]:
reader = PdfReader('apple_event_brief.pdf')
print(len(reader.pages))
last_page = (len(reader.pages))
page = reader.pages[0]
text = page.extract_text()
print(text)

262
Page1of262 ©2023Factiva,Inc.Allrightsreserved.
EventBriefofQ12023AppleIncEarningsCall-Final...................................................................................................2
EventBriefofQ42022AppleIncEarningsCall-Final.................................................................................................17
EventBriefofQ32022AppleIncEarningsCall-Final.................................................................................................30
EventBriefofQ22022AppleIncEarningsCall-Final.................................................................................................44
EventBriefofQ12022AppleIncEarningsCall-Final.................................................................................................56
EventBriefofQ42021AppleIncEarningsCall-Final.................................................................................................68
EventBriefofQ32021AppleIncEarningsCall-Final........................................

In [37]:
text = text.split('\n')

In [38]:
text

['Page1of262 ©2023Factiva,Inc.Allrightsreserved.',
 'EventBriefofQ12023AppleIncEarningsCall-Final...................................................................................................2',
 'EventBriefofQ42022AppleIncEarningsCall-Final.................................................................................................17',
 'EventBriefofQ32022AppleIncEarningsCall-Final.................................................................................................30',
 'EventBriefofQ22022AppleIncEarningsCall-Final.................................................................................................44',
 'EventBriefofQ12022AppleIncEarningsCall-Final.................................................................................................56',
 'EventBriefofQ42021AppleIncEarningsCall-Final.................................................................................................68',
 'EventBriefofQ32021AppleIncEarningsCall-Final..............

In [39]:
pdf_pages = pd.DataFrame(columns=['name','first_page'])

In [40]:
# text = text.split("\n")
for i in range(1,len(text)):
    value = re.split('\.+', text[i])
    test = pd.DataFrame({'name':[value[0]],'first_page':[int(value[1])]})
    pdf_pages = pd.concat([pdf_pages,test])

In [41]:
pdf_pages['last_page'] = pdf_pages['first_page'].shift(-1)
pdf_pages.iloc[len(pdf_pages)-1,2] = last_page + 1
pdf_pages['last_page'] = pdf_pages['last_page'] - 1
pdf_pages

,name,first_page,last_page
0,EventBriefofQ12023AppleIncEarningsCall-Final,2,16
0,EventBriefofQ42022AppleIncEarningsCall-Final,17,29
0,EventBriefofQ32022AppleIncEarningsCall-Final,30,43
0,EventBriefofQ22022AppleIncEarningsCall-Final,44,55
0,EventBriefofQ12022AppleIncEarningsCall-Final,56,67
0,EventBriefofQ42021AppleIncEarningsCall-Final,68,81
0,EventBriefofQ32021AppleIncEarningsCall-Final,82,95
0,EventBriefofQ22021AppleIncEarningsCall-Final,96,109
0,EventBriefofQ12021AppleIncEarningsCall-Final,110,123
0,EventBriefofQ42020AppleIncEarningsCall-Final,124,137


In [42]:
for i in range(len(pdf_pages)):
    writer = PdfWriter()
    page_range = range(pdf_pages.iloc[i,1], pdf_pages.iloc[i,2] + 1)

    for page_num, page in enumerate(reader.pages, 1):
        if page_num in page_range:
            writer.add_page(page)
    
    file_name = "files/" + pdf_pages.iloc[i,0]
    with open(f'{file_name}.pdf', 'wb') as out:
        writer.write(out)